In [ ]:
# Install the BM3D package (if not already installed)
!pip install bm3d

# Import Libraries
import bm3d
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from skimage import io

# Paths to input, expected, and output folders
input_folder = 'test/sar_tif'        # Folder containing noisy images
expected_folder = 'test/gray_tif'    # Folder with expected images for comparison
output_folder = 'BM3D_outputs'       # Folder to save denoised images

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Set the sigma_psd parameter for tuning
sigma_psd = 0.1  # Adjust based on noise level; try values between 0.05 and 0.2

# Helper functions
def snr(img, noise_img):
    signal = np.mean(img) ** 2
    noise = np.mean((img - noise_img) ** 2)
    return 10 * np.log10(signal / noise) if noise != 0 else float('inf')

def enl(img):
    mean = np.mean(img)
    std = np.std(img)
    return (mean ** 2) / (std ** 2) if std != 0 else float('inf')

# Lists to store metric values for averaging
psnr_values = []
ssim_values = []
snr_values = []
enl_values = []

# Process each image in the input folder
for filename in os.listdir(input_folder):
    # Ensure we process only .tif files
    if not filename.endswith('.tif'):
        continue
    
    # Construct file paths
    input_path = os.path.join(input_folder, filename)
    expected_path = os.path.join(expected_folder, filename)
    output_path = os.path.join(output_folder, filename)

    # Load the noisy image in .tif format
    image = io.imread(input_path)
    if image is None:
        print(f"Error: Image {filename} not found or path is incorrect.")
        continue

    # Load the expected image for comparison
    expected_image = io.imread(expected_path)
    if expected_image is None:
        print(f"Warning: Expected image {filename} not found.")
        continue

    # Normalize images to [0,1] range
    image = image.astype(np.float32) / 255.0
    expected_image = expected_image.astype(np.float32) / 255.0

    # Apply BM3D despeckling
    denoised_image = bm3d.bm3d(image, sigma_psd)

    # Save the denoised image in the output folder as .tif
    io.imsave(output_path, (denoised_image * 255).astype(np.uint8))

    # Calculate metrics
    psnr_values.append(psnr(expected_image, denoised_image))
    ssim_values.append(ssim(expected_image, denoised_image, data_range=denoised_image.max() - denoised_image.min()))
    snr_values.append(snr(expected_image, denoised_image))
    enl_values.append(enl(denoised_image))

# Calculate average values of the metrics
avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)
avg_snr = np.mean(snr_values)
avg_enl = np.mean(enl_values)

# Display the average metrics
print("Average PSNR:", avg_psnr)
print("Average SSIM:", avg_ssim)
print("Average SNR:", avg_snr)
print("Average ENL:", avg_enl)

print("Denoising complete for all images. Denoised images saved to the output folder.")


C:\Users\newhi\AppData\Local\Temp\ipykernel_21128\3966325923.py:75: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  psnr_values.append(psnr(expected_image, denoised_image))
